### Imports

In [1]:
import pandas as pd

In [2]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, save
from bokeh.models import Label, Row, HoverTool, ColumnDataSource, Whisker, BoxZoomTool, ResetTool, PanTool, SaveTool

output_notebook()

Loading BokehJS ...

### Load the summary data

In [3]:
q_and_s_summary_df = pd.read_csv('[your_path]/all_bound_planet_q_s.csv', index_col=['data_challenge_lc_number'])

In [4]:
q_and_s_summary_df

,true_q,true_s,LS_q,LS_q_err,LS_s,LS_s_err,LS_chi2,LX_q,LX_q_err,LX_s,LX_s_err,LX_chi2,LO_q,LO_q_err,LO_s,LO_s_err,LO_chi2
data_challenge_lc_number,,,,,,,,,,,,,,,,,
4,0.010920,2.481240,0.010135,4.788402e-04,2.614823,0.074210,10776.265086,0.010135,2.353786e-03,2.614823,0.052744,10776.259082,0.010135,4.700959e-03,2.614823,0.085485,10776.259095
8,0.000692,1.921590,0.000668,7.369358e-05,2.106614,0.061153,8693.730639,0.000668,1.446106e-04,2.105069,0.094812,8692.418840,0.000667,1.978680e-04,2.105064,0.110770,8692.393028
12,0.000025,1.032940,0.000024,7.824279e-07,1.035088,0.000954,6331.995117,0.000024,1.144112e-06,1.035224,0.001406,6331.544912,0.000024,1.538576e-06,1.035224,0.007361,6331.544911
25,0.000066,1.088030,0.000092,2.343772e-05,1.095779,0.002609,7641.429091,0.000094,2.401100e-05,1.093012,0.002288,7579.194170,0.000094,2.917077e-05,1.093020,0.043553,7578.830621
32,0.000031,0.735662,0.000027,1.480027e-06,0.739486,0.000797,6220.875651,0.000026,1.653699e-06,0.739362,0.000971,6195.367353,0.000026,2.062042e-06,0.739377,0.003396,6195.151217
40,0.000758,0.393097,0.000025,2.312082e-06,2.605803,0.000503,7537.741245,0.000028,1.823073e-06,2.462879,0.000337,7516.467562,0.000028,8.903933e-07,2.462881,0.000467,7516.241459
47,0.000008,1.027670,0.000010,1.274968e-06,1.036000,0.008991,6329.311033,0.000010,1.465114e-06,0.932922,0.010172,6328.791941,0.000010,1.490264e-06,0.931861,0.012811,6328.300265
50,0.000241,0.540982,0.000205,3.669808e-05,0.624170,0.025123,10176.383441,0.000159,6.394785e-05,0.589812,0.034676,10163.911122,0.000159,8.566293e-05,0.590107,0.040813,10163.698057
53,0.000856,2.191240,0.003756,5.925514e-04,0.559352,0.007503,12925.384467,0.003754,5.615880e-04,0.559431,0.003227,12921.933581,0.003751,3.933651e-04,0.559412,0.013820,12921.610265


### Defining plotting functions

In [5]:
def plotting_mass_ratio(model_type, x_range=None, y_range=None):
    if model_type =='LS':
        title = f'Mass-ratio - 2L1S ({model_type})'
    elif model_type == 'LX':
        title = f'Mass-ratio - 2L1S+parallax ({model_type})'
    elif model_type == 'LO':
        title = f'Mass-ratio - 2L1S+parallax+OM ({model_type})'    
    else:
        raise ValueError(f'model_type {model_type} should be LS, LX or LO')
    source = ColumnDataSource(data=dict(
    x=q_and_s_summary_df['true_q'],
    y=q_and_s_summary_df[f'{model_type}_q'],
    upper=q_and_s_summary_df[f'{model_type}_q']+q_and_s_summary_df[f'{model_type}_q_err'],   
    lower=q_and_s_summary_df[f'{model_type}_q']-q_and_s_summary_df[f'{model_type}_q_err'],
    desc=q_and_s_summary_df.index,
    ))
    hover = HoverTool(tooltips=[
        ("(true,model)", "($x, $y)"),
        ("LC#", "@desc")
    ])
    if x_range is None:
        p = figure(width=350, height=350, title=title, x_axis_label='true q', y_axis_label=f'{model_type} q', tools=[hover, BoxZoomTool(), ResetTool(), PanTool(), SaveTool()])
    else:
        p = figure(width=350, height=350, title=title, x_axis_label='true q', y_axis_label=f'{model_type} q', tools=[hover, BoxZoomTool(), ResetTool(), PanTool(), SaveTool()], x_range=x_range, y_range=y_range)
    p = figure(width=350, height=350, title=title, x_axis_label='true q', y_axis_label=f'{model_type} q', tools=[hover, BoxZoomTool(), ResetTool(), PanTool(), SaveTool()])
    p.line([0, 0.025], [0, 0.025], line_width=1, line_color='red')
    p.scatter('x', 'y', size=5, fill_alpha=0.6, source=source)
    whisker_errorbar = Whisker(source=source, base="x", upper="upper", lower="lower",
                               line_width=1.0, line_alpha=1.0)  # level="overlay",
    p.add_layout(whisker_errorbar)
    # p.scatter(q_and_s_summary_df['true_q'][107], q_and_s_summary_df[f'{model_type}_q'][107], size=5, fill_alpha=0.6, fill_color='red')
    # p.scatter(q_and_s_summary_df['true_q'][226], q_and_s_summary_df[f'{model_type}_q'][226], size=5, fill_alpha=0.6, fill_color='red')
    discrepancy = [107, 226]
    for event in discrepancy:
        citation = Label(x=q_and_s_summary_df['true_q'][event] + 0.001, y=q_and_s_summary_df[f'{model_type}_q'][event] - 0.025, text=f'LC# {event}', text_font_size='12px')
        p.add_layout(citation)
    if model_type == 'LO':
        citation_3 = Label(x=q_and_s_summary_df['true_q'][214] + 0.001, y=q_and_s_summary_df[f'{model_type}_q'][214] - 0.025, text=f'LC# {214}', text_font_size='12px')
        p.add_layout(citation_3)
    return p

In [6]:
def plotting_separation(model_type, x_range=None, y_range=None):
    if model_type =='LS':
        title = f'Separation - 2L1S ({model_type})'
    elif model_type == 'LX':
        title = f'Separation - 2L1S+parallax ({model_type})'
    elif model_type == 'LO':
        title = f'Separation - 2L1S+parallax+OM ({model_type})'    
    else:
        raise ValueError(f'model_type {model_type} should be LS, LX or LO')
    source = ColumnDataSource(data=dict(
    x=q_and_s_summary_df['true_s'],
    y=q_and_s_summary_df[f'{model_type}_s'],
    upper=q_and_s_summary_df[f'{model_type}_s']+q_and_s_summary_df[f'{model_type}_s_err'],   
    lower=q_and_s_summary_df[f'{model_type}_s']-q_and_s_summary_df[f'{model_type}_s_err'],
    desc=q_and_s_summary_df.index,
    ))
    hover = HoverTool(tooltips=[
        ("(true,model)", "($x, $y)"),
        ("LC#", "@desc")
    ])
    if x_range is None:
        p = figure(width=350, height=350, title=title, x_axis_label='true s', y_axis_label=f'{model_type} s', tools=[hover, BoxZoomTool(), ResetTool(), PanTool(), SaveTool()])
    else:
       p = figure(width=350, height=350, title=title, x_axis_label='true s', y_axis_label=f'{model_type} s', tools=[hover, BoxZoomTool(), ResetTool(), PanTool(), SaveTool()], x_range=x_range, y_range=y_range) 
    # add a circle renderer with vectorized colors and sizes
    p.line([0, 5.5], [0, 5.5], line_width=1, line_color='red')
    p.scatter('x', 'y', size=5, fill_alpha=0.6, source=source)
    whisker_errorbar = Whisker(source=source, base="x", upper="upper", lower="lower",
                               line_width=1.0, line_alpha=1.0)  # level="overlay",
    p.add_layout(whisker_errorbar)
    return p

## Mass ratio

In [7]:
p1 = plotting_mass_ratio(model_type = 'LS')
p2 = plotting_mass_ratio(model_type = 'LX', x_range=p1.x_range, y_range=p1.y_range)
p3 = plotting_mass_ratio(model_type = 'LO', x_range=p1.x_range, y_range=p1.y_range)
show(Row(p1, p2, p3))

## Separation

In [8]:
p1 = plotting_separation(model_type = 'LS')
p2 = plotting_separation(model_type = 'LX', x_range=p1.x_range, y_range=p1.y_range)
p3 = plotting_separation(model_type = 'LO', x_range=p1.x_range, y_range=p1.y_range)
show(Row(p1, p2, p3))